# Trabajo Integrador
Realizar un Scraping de la página web de cúspide libros, www.cuspide.com , y de allí obtener el 
listado de los 100 libros más vendidos de la semana. 
* Los datos deben ser: 
1. título del libro 
2. url del libro
3. precio en pesos
4. precio en dólares considerando la cotización del dólar blue de Argentina o en Euros 
5. fecha. 
+ La cotización del dólar blue de Argentina o Euros debe ser obtenida también utilizando scraping de cualquier página web a elección. 
Estos datos deben ser reflejados en un archivo .csv con los respectivos campos. Asimismo debe 
crearse otro archivo de salida como log de errores, por si algún título no se puede scrapear por 
alguna razón, con la información del título del libro en cuestión, y la url del mismo.

In [6]:
#Importamos las librerias a utilizar
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests as r
from datetime import datetime

0.0


In [8]:
#Probamos en solicitar la informacion a la pagina web
requisito = r.get('https://cuspide.com/100-mas-vendidos')
# requisito.status_code # si da 200 significa que entro satisfactoriamente a la pagina web

#extraimos la informacion en general de la pagina web
contenido_html = requisito.text
Pagina_Principal = bs(contenido_html, 'html.parser')

#extraimos las diviciones de todos los libros, cosa que nos va a ayudar a que tengamos todos los datos de cada libro y ir extrayendo lo necesario
lista_libros = Pagina_Principal.find_all('div',class_="col small-6 large-2 max-w-large")

#sacamos informacion de la pagina del cronista para saber el dolar blue
ingreso_dolar_blue = r.get('https://www.cronista.com/MercadosOnline/dolar.html')
contenido_dolar_blue = ingreso_dolar_blue.text
pagina_dolar_blue = bs(contenido_dolar_blue, 'html.parser')
pagina_adentro_dolar_blue = pagina_dolar_blue.find_all('div', class_='sell-value')
valor_dolar_blue= pagina_adentro_dolar_blue[1].text
valor_dolar_blue= float(valor_dolar_blue.replace('$', '').replace('.', '').replace(',','.'))

#guardamos el dia en el que estamos
fecha_hoy = datetime.now().date()

#creamos la variable
nombres_libros = []
url_libros = []
url_libros_fallando = []
Precio_pesos = []
precio_dolares = []
fecha = []
Preciodolar_blue = []


for nombres in lista_libros:
        #extraemos el nombre del libro de la pagina de top100 y lo guardamos en una lista
        nombre_libro = nombres.find('h3').text
        nombres_libros.append(nombre_libro)

        #extraemos la url
        url = nombres.find_all('a')
        
        if url:
                h_url = url[0].get('href')
                
        #ingresamos a la pagina del libro
        ingreso = r.get(h_url)
        if ingreso.status_code == 200:
                url_libros.append(h_url)
                contenido_html_libro = ingreso.text
                pagina_Libro = bs(contenido_html_libro, 'html.parser')
                pagina_Libro_adentro = pagina_Libro.find('div', class_='product-container')
        else: url_libros_fallando.append(h_url)
        
        #extraemos el precio en pesos por cada libro    
        preciopesos = pagina_Libro_adentro.find('bdi')
        if preciopesos:
                preciopesos = preciopesos.text
                preciopesos = preciopesos.replace('$\xa0', '').replace('.', '').replace(',', '.')
                preciopesos = float(preciopesos)
        else:
                preciopesos = 0
        
        #extraemos el precio en dolares
        preciodolar = pagina_Libro_adentro.find('span', style='font-size: 1.3em')
        if preciodolar:
                preciodolar = float(preciodolar.text.strip().replace(',', '.'))
        else:
                preciodolar = 0
        
        #hacemos la conversion de pesos a dolar blue
        if preciopesos != 0:
                precio_dolar_blue_final = round(preciopesos/valor_dolar_blue, 2)
                Preciodolar_blue.append(precio_dolar_blue_final)
        else: Preciodolar_blue.append(0)
        
        #guardamos el dia en el que se actualizó la info
        fecha.append(fecha_hoy.strftime('%d/%m/%Y'))
        
        #guardamos los precios en sus respectivas listas
        Precio_pesos.append(preciopesos)
        precio_dolares.append(preciodolar)





80.04072451591492


In [16]:
tabla = pd.DataFrame({'nombre de libros':nombres_libros , 
                      'url del libro':url_libros,
                      'precio del libro en pesos': Precio_pesos,
                      'precio del libro en dolares':precio_dolares,
                       'precio dolar en dolar blue':Preciodolar_blue,
                      'fecha': fecha
                      })
lista = pd.Series({'url fallados':url_libros_fallando})
tabla.to_csv('InfoTOP100.csv',sep=',',index=False)
lista.to_csv('logErrores.csv',sep='|',index=False)